In [1]:
!pip install mediapipe opencv-python

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\vibhu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_internal\utils\logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "C:\Users\vibhu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_vendor\rich\console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "C:\Users\vibhu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_vendor\rich\console.py", line 1305, in render
    for render_output in iter_render:
  File "C:\Users\vibhu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_internal\utils\logging.py", line 134, in _

In [2]:
!pip install imageio

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\vibhu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_internal\utils\logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "C:\Users\vibhu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_vendor\rich\console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "C:\Users\vibhu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_vendor\rich\console.py", line 1305, in render
    for render_output in iter_render:
  File "C:\Users\vibhu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pip\_internal\utils\logging.py", line 134, in _

In [2]:
import glob
import cv2
import mediapipe as mp
import numpy as np
import imageio
from PIL import Image 

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [19]:
reader = cv2.VideoCapture('C:/Users/vibhu/Desktop/projects/pose_detection/test2.mp4')
frame_width = int(reader.get(3))
frame_height = int(reader.get(4))
frame_size = (frame_width,frame_height)
fps = 20
output = cv2.VideoWriter('C:/Users/vibhu/Desktop/projects/pose_detection/output6.mp4', cv2.VideoWriter_fourcc(*'DIVX'), fps,frame_size)

In [6]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [18]:
counter = 6
stage = None
timer=0
while(reader.isOpened()):

  # Curl counter variables

  with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
    ret, frame = reader.read()
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
        
    # Make detection
    results = pose.process(image)
      
    # Recolor back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Extract landmarks
    try:
      landmarks = results.pose_landmarks.landmark
      hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
      knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
      ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

      angle=calculate_angle(hip,knee,ankle)

      image=cv2.putText(image, str(angle), 
                           tuple(np.multiply(knee, [854, 640]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
      
      if angle > 150:
        stage = "down"
        timer=0

      if angle < 140 and stage =='down':
        stage="up"

      if stage=='up':
        timer+=0.05
        #putting timer
        # cv2.rectangle(image, (0,80), (225,153), (0,0,0), -1)

        # cv2.putText(image, 'Timer', (3,73),  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        # cv2.putText(image, int(timer),   (200,225),  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
      
      if stage=='up' and angle>140:
        if int(timer)>=7:
          counter +=1
          print(counter,'rep --', int(timer),'seconds')
          timer=0
        stage='down'
        timer=0

    except:
      pass 
            
    # Render curl counter
    # Setup status box
    cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
       
    # Rep data
    cv2.putText(image, 'REPS', (15,12), 
              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(image, str(counter), 
             (10,60), 
           cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
    # Stage data
    cv2.putText(image, 'STAGE', (65,12), 
                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.putText(image, stage, 
              (60,60), 
              cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
    

    
    # Render detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                  )               
          
    output.write(image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

7 rep -- 34 seconds
8 rep -- 26 seconds
9 rep -- 31 seconds
10 rep -- 39 seconds


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
